In [33]:
import pandas as pd
import numpy as np
import os
import math

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import tqdm

import julia

In [34]:
wkdir = "/Users/chrisolen/Documents/uchicago_courses/deep_learning_and_image_recognition/finance/fin-portfolio-mvo/"
data_files = os.listdir(wkdir+'data')
data_files.remove('.DS_Store')


In [35]:
data = pd.read_csv(wkdir+'data/'+'data_cleaned.csv')

In [36]:
data.columns

Index(['date', 'UST_10YR', 'UIVE_SP500VALUEETF', 'VNQ_VANGREALEST', 'USFFR',
       'EMB_USDEMRGBOND', 'LQD_CORPBOND', 'MUB_MUNIBOND', 'SHY_1-3USTR',
       'USDJPY', 'USDGBP', 'VIG_VANGDIV', 'IVV_SP500', 'USDRMB', 'CRUDOIL',
       'CFE_VIX', 'EEM_MSCIEMERGING', 'USDEUR', 'XLE_ENERGYSPDR', 'SP500_GSCI',
       'EFA_MSCIEAFE', 'TIP_TIPSBOND', 'UST_2YR', 'USDOIS', 'CHNGDP', 'USGDP',
       'EZGDP', 'US_UNEMP', 'CHNGDP_Shock', 'USGDP_Shock', 'EZGDP_Shock',
       'US_UNEMP_Shock', 'VNQ_VOL', 'EMB_VOL', 'LQD_VOL', 'MUB_VOL', 'VIG_VOL',
       'IVV_VOL', 'EEM_VOL', 'EFA_VOL', 'XLE_VOL', 'SHY_VOL', 'TIP_VOL'],
      dtype='object')

In [37]:
# Separating features into macroeconomic indictors and portfolio:

econ = ['CHNGDP','USGDP','EZGDP','US_UNEMP']

shock = ['CHNGDP_Shock','USGDP_Shock','EZGDP_Shock','US_UNEMP_Shock']

finstruments = ['UST_10YR','USFFR','USDRMB','CRUDOIL','CFE_VIX','USDEUR','UST_2YR',
             'SP500_GSCI','USDOIS','UIVE_SP500VALUEETF','USDJPY','USDGBP']

assets = ['VNQ_VANGREALEST','EMB_USDEMRGBOND','LQD_CORPBOND',
            'MUB_MUNIBOND','SHY_1-3USTR','VIG_VANGDIV','IVV_SP500','EEM_MSCIEMERGING',
            'XLE_ENERGYSPDR','EFA_MSCIEAFE','TIP_TIPSBOND']

asset_vols = data.columns[-11:]

In [38]:
# Create response variable matrix to be subsequently transformed into y

portfolio = data[assets]
log_returns = np.log(portfolio/portfolio.shift(1)).dropna()
log_returns

,VNQ_VANGREALEST,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,VIG_VANGDIV,IVV_SP500,EEM_MSCIEMERGING,XLE_ENERGYSPDR,EFA_MSCIEAFE,TIP_TIPSBOND
1,0.017507,-0.000754,-0.002072,0.002550,-0.000484,0.008526,0.013131,0.032254,0.013098,0.012923,-0.000748
2,0.015289,0.000881,-0.002946,-0.001470,0.000000,0.008820,0.006994,0.011717,-0.004173,-0.007582,-0.004498
3,0.000174,-0.003646,0.007275,0.001274,0.002780,-0.014002,-0.012183,-0.028531,-0.012227,-0.021928,0.007393
4,-0.004192,0.003451,-0.004140,0.004496,0.000362,0.006104,0.010139,0.014602,0.019647,0.007749,0.004187
5,-0.020523,0.001662,0.005359,0.005446,0.000844,-0.018049,-0.025074,-0.046287,-0.042255,-0.032558,0.005093
...,...,...,...,...,...,...,...,...,...,...,...
2954,-0.004231,-0.000178,-0.000393,0.001754,0.001061,-0.015283,-0.018066,-0.007669,-0.024861,-0.022777,0.000000
2955,0.009630,0.004793,0.003922,0.002101,0.002119,0.005662,0.008287,0.012830,0.010756,0.004675,0.003176
2956,0.006012,0.008728,0.004140,0.000000,-0.000118,0.013426,0.013536,0.004403,0.006294,0.007357,0.003080
2957,-0.002143,-0.005634,-0.005080,-0.000787,-0.001059,-0.005502,-0.004503,-0.008333,-0.008928,0.001927,-0.002738


In [39]:
# Create features matrix to be subsequently transformed into X

# Log returns of assets
asset_features = np.log(data[assets]/data[assets].shift(1))

# Scaled macroeconomic factors
scale = StandardScaler()
econ_features = pd.DataFrame(scale.fit_transform(data[econ]), columns = data[econ].columns)

# Log returns of financial instruments
finstruments_features = np.log(data[finstruments]/data[finstruments].shift(1))

features = pd.concat([asset_features, econ_features, finstruments_features, 
                     data[shock], data[asset_vols]], axis = 1).dropna()

features

,VNQ_VANGREALEST,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,VIG_VANGDIV,IVV_SP500,EEM_MSCIEMERGING,XLE_ENERGYSPDR,EFA_MSCIEAFE,...,EMB_VOL,LQD_VOL,MUB_VOL,VIG_VOL,IVV_VOL,EEM_VOL,EFA_VOL,XLE_VOL,SHY_VOL,TIP_VOL
1,0.017507,-0.000754,-0.002072,0.002550,-0.000484,0.008526,0.013131,0.032254,0.013098,0.012923,...,0.234243,0.345803,0.124218,0.506379,1.997616,0.789580,0.301695,1.721589,0.073689,0.034641
2,0.015289,0.000881,-0.002946,-0.001470,0.000000,0.008820,0.006994,0.011717,-0.004173,-0.007582,...,0.206426,0.342953,0.109681,0.364033,1.117511,0.986725,0.128841,0.637754,0.044497,0.228210
3,0.000174,-0.003646,0.007275,0.001274,0.002780,-0.014002,-0.012183,-0.028531,-0.012227,-0.021928,...,0.162948,0.319982,0.105214,0.394360,1.159634,0.886779,0.290207,0.671141,0.115845,0.287785
4,-0.004192,0.003451,-0.004140,0.004496,0.000362,0.006104,0.010139,0.014602,0.019647,0.007749,...,0.156563,0.286197,0.266496,0.358497,1.143866,0.835776,0.302704,0.642441,0.128374,0.452968
5,-0.020523,0.001662,0.005359,0.005446,0.000844,-0.018049,-0.025074,-0.046287,-0.042255,-0.032558,...,0.191924,0.368284,0.472737,0.516701,1.517778,1.132523,0.545225,1.298218,0.154045,0.688876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,-0.004231,-0.000178,-0.000393,0.001754,0.001061,-0.015283,-0.018066,-0.007669,-0.024861,-0.022777,...,0.507070,0.183439,0.079183,1.388910,3.723606,0.336348,0.355992,1.403093,0.052249,0.066858
2955,0.009630,0.004793,0.003922,0.002101,0.002119,0.005662,0.008287,0.012830,0.010756,0.004675,...,0.448687,0.258882,0.157575,1.382292,3.616485,0.232056,0.379803,1.379554,0.112472,0.190971
2956,0.006012,0.008728,0.004140,0.000000,-0.000118,0.013426,0.013536,0.004403,0.006294,0.007357,...,0.650769,0.420868,0.189473,1.318283,3.465087,0.277993,0.350471,1.049843,0.125579,0.332310
2957,-0.002143,-0.005634,-0.005080,-0.000787,-0.001059,-0.005502,-0.004503,-0.008333,-0.008928,0.001927,...,0.642051,0.415175,0.187563,0.907981,2.591067,0.260615,0.227662,0.535416,0.114761,0.308982


In [40]:
# Model definition:

batch_size = 5
n_batches = math.floor(features.shape[0] / batch_size) # number of batches in an epoch
n_timesteps = 10 # length of series used for prediction (i.e. how many days we're predict off of)
n_features = features.shape[1] # number of features used for prediction
n_predicted_vars = log_returns.shape[1] # number of asset values being predicted
n_epochs = 10
look_ahead_time = 1 # number of days in advance we will predict
validation_split = 0.1
checkpoint_dir = "model_checkpoints"

burn_in_length = 200
burn_in_epochs = 50

delta = np.array([1.5])
min_weight = np.array([0.01])
vol_window = 10

daily_portfolio_weights = []


In [41]:
# Sliding window:
            
def sliding_window(features, log_returns, end_index, n_timesteps, look_ahead_time):
    
    """
    Takes in features matrix and response var (i.e. log_returns) matrix
    RETURNS: X and y matrices for training
    
    end_index: the most recent time instance used for prediction
    n_timesteps: length of series used for prediction
    look_ahead_time: how many days ahead we are predicting
    """
    
    # start index is derived from end_index - n_timesteps
    # X slices up to but does not include end_index
    X = np.array(features.iloc[(end_index-n_timesteps):end_index, :])                              
    
    # y includes end_index and slices up to but does not include end_index + look_ahead_time
    # thus, if look_ahead_time = 1, y will only include one day for prediction
    y = np.array(log_returns.iloc[end_index+look_ahead_time-1, :]) # currently can only make predictions of a single time step
    
    # return (X:[n_timesteps,n_features], y:[look_ahead_time, n_assets])
    return X, y
    
    

In [42]:
def generate_epoch(features, log_returns, n_timesteps, look_ahead_time):
    
    # begin with empty arrays to which we will append 
    X = np.array([]) 
    y = np.array([])
    
    window_count = 0
    
    for i in range(len(features)-n_timesteps):
        
        end_index = i + n_timesteps
        
        # pull out one window
        X_one, y_one = sliding_window(features, log_returns, end_index, n_timesteps, look_ahead_time)
        
        ### append sliding windows ###
        # append X_one:[n_timesteps,n_features] to batch ndarray X
        X = np.append(X, X_one)
        # append y_one:[look_ahead_time, n_assets] to batch ndarray y
        y = np.append(y, y_one)
        
        # count the number of windows (i.e. training instances)
        window_count += 1
     
    
    # reshape training vectors given window_count
    X = X.reshape(window_count, n_timesteps, features.shape[1])    
    y = y.reshape(window_count, log_returns.shape[1])
        
    return X, y
        
   

In [43]:
# build model:

sliding_window_input = tf.keras.layers.Input(shape=(n_timesteps, n_features,), 
                                             name = "input_layer")
lstm_out = tf.keras.layers.LSTM(n_predicted_vars, 
                                activation='tanh', recurrent_activation='sigmoid',
                                dropout=0.2, stateful=False,
                                name = "lstm")(sliding_window_input)
dense_out = tf.keras.layers.Dense(n_predicted_vars, 
                                  activation='relu', name = "dense_layer")(lstm_out)

model = tf.keras.models.Model(inputs=sliding_window_input, outputs=dense_out)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss='mse')    
model.summary()

checkpoints = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir + "/saved_model.hdf5", verbose=1, 
                                                     save_best_only=True, 
                                                     mode='auto', 
                                                     save_freq='epoch')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 10, 42)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 11)                2376      
_________________________________________________________________
dense_layer (Dense)          (None, 11)                132       
Total params: 2,508
Trainable params: 2,508
Non-trainable params: 0
_________________________________________________________________


In [44]:
# burn in period

training_features = features.iloc[0:burn_in_length,:]
training_response = log_returns.iloc[0:burn_in_length,:]

X, y = generate_epoch(training_features, training_response, n_timesteps, look_ahead_time)

model.fit(X, y,
          validation_split = validation_split,
          epochs = burn_in_epochs,
          batch_size = batch_size,
          callbacks = [checkpoints])
    
    

Train on 171 samples, validate on 19 samples
Epoch 1/50
125/171 [====================>.........] - ETA: 0s - loss: 0.0334
Epoch 00001: val_loss improved from inf to 0.05262, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 2s 9ms/sample - loss: 0.0329 - val_loss: 0.0526
Epoch 2/50
130/171 [=====================>........] - ETA: 0s - loss: 0.0323
Epoch 00002: val_loss improved from 0.05262 to 0.05159, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 977us/sample - loss: 0.0316 - val_loss: 0.0516
Epoch 3/50
145/171 [========================>.....] - ETA: 0s - loss: 0.0318
Epoch 00003: val_loss improved from 0.05159 to 0.05057, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 925us/sample - loss: 0.0321 - val_loss: 0.0506
Epoch 4/50
145/171 [========================>.....] - ETA: 0s - loss: 0.0290
Epoch 00004: val_loss improved from 0.05057 to 0.0496

Epoch 31/50
160/171 [===========================>..] - ETA: 0s - loss: 0.0133
Epoch 00031: val_loss improved from 0.03321 to 0.03281, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 827us/sample - loss: 0.0133 - val_loss: 0.0328
Epoch 32/50
155/171 [==========================>...] - ETA: 0s - loss: 0.0126
Epoch 00032: val_loss improved from 0.03281 to 0.03246, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 834us/sample - loss: 0.0123 - val_loss: 0.0325
Epoch 33/50
155/171 [==========================>...] - ETA: 0s - loss: 0.0128
Epoch 00033: val_loss improved from 0.03246 to 0.03213, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 838us/sample - loss: 0.0128 - val_loss: 0.0321
Epoch 34/50
155/171 [==========================>...] - ETA: 0s - loss: 0.0122
Epoch 00034: val_loss improved from 0.03213 to 0.03179, saving model to model_checkpoint

In [45]:
# day-to-day training and prediction

# initialize julia instances
j = julia.Julia(compiled_modules=False)

# iterate through the remaining time steps
for i in tqdm.tqdm(range(len(features)-burn_in_length)):
    
    # extract training features for the current time-step (includes all instances up to the current time-step)
    training_features = features.iloc[0:burn_in_length+i,:]
    # extract response variables (log returns) for the current time-step
    training_response = log_returns.iloc[0:burn_in_length+i,:]
    
    # generate training epoch of sliding windows for current time-step
    X, y = generate_epoch(training_features, training_response, n_timesteps, look_ahead_time)
    
    # load latest model weights
    model = tf.keras.models.load_model(wkdir + "model_checkpoints/saved_model.hdf5")
    
    # fit to features for current time-step
    model.fit(X, y,
          validation_split = validation_split,
          epochs = burn_in_epochs,
          batch_size = batch_size,
          callbacks = [checkpoints])
    
    # extract most recent window of features for current time-step prediction
    pred_window = np.array(features.iloc[-n_timesteps:,:]).reshape(1,n_timesteps,features.shape[1])
    # extract log returns for the last 'vol_window' time-steps for current 'vol_window'-day volatility
    returns_vol_window = np.array(log_returns.iloc[-vol_window:,:])
    
    # predict mu for tomorrow
    mu = model.predict(pred_window)
    # find current sigma 
    sigma = np.dot(np.transpose(returns_vol_window),returns_vol_window)
    
    # save optimization params to tmp
    np.savetxt("tmp/mu.txt",mu)
    np.savetxt("tmp/sigma.txt",sigma)
    np.savetxt("tmp/delta.txt",delta)
    np.savetxt("tmp/min_weight.txt",min_weight)
    
    # run mvo julia optimizer
    %time j.include("mvo.jl")
    
    # pull in optimal weights from optimizer for rebalancing
    with open("tmp/weights.txt", 'r') as f:
        w = f.readlines()
        weights = [float(e.replace('\n',"")) for e in w]
    
    # append rebalanced weights to daily_portfolio_weights object
    daily_portfolio_weights.append(weights)
    
    
    

  0%|          | 0/2758 [00:00<?, ?it/s]

Train on 171 samples, validate on 19 samples
Epoch 1/50
120/171 [====================>.........] - ETA: 0s - loss: 0.0108
Epoch 00001: val_loss improved from 0.02727 to 0.02696, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 1s 5ms/sample - loss: 0.0100 - val_loss: 0.0270
Epoch 2/50
135/171 [======================>.......] - ETA: 0s - loss: 0.0100
Epoch 00002: val_loss improved from 0.02696 to 0.02653, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 948us/sample - loss: 0.0098 - val_loss: 0.0265
Epoch 3/50
140/171 [=======================>......] - ETA: 0s - loss: 0.0096
Epoch 00003: val_loss improved from 0.02653 to 0.02613, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 940us/sample - loss: 0.0097 - val_loss: 0.0261
Epoch 4/50
145/171 [========================>.....] - ETA: 0s - loss: 0.0107
Epoch 00004: val_loss improved from 0.02613 to 0.

Epoch 31/50
160/171 [===========================>..] - ETA: 0s - loss: 0.0061
Epoch 00031: val_loss improved from 0.01892 to 0.01873, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 816us/sample - loss: 0.0060 - val_loss: 0.0187
Epoch 32/50
165/171 [===========================>..] - ETA: 0s - loss: 0.0064
Epoch 00032: val_loss improved from 0.01873 to 0.01851, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 812us/sample - loss: 0.0064 - val_loss: 0.0185
Epoch 33/50
160/171 [===========================>..] - ETA: 0s - loss: 0.0066
Epoch 00033: val_loss improved from 0.01851 to 0.01826, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 812us/sample - loss: 0.0065 - val_loss: 0.0183
Epoch 34/50
155/171 [==========================>...] - ETA: 0s - loss: 0.0053
Epoch 00034: val_loss improved from 0.01826 to 0.01807, saving model to model_checkpoint

/Users/chrisolen/anaconda3/lib/python3.7/site-packages/julia/core.py:689: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  FutureWarning,
  0%|          | 1/2758 [00:09<7:12:46,  9.42s/it]

CPU times: user 96.9 ms, sys: 3.76 ms, total: 101 ms
Wall time: 112 ms
Train on 171 samples, validate on 20 samples
Epoch 1/50
135/171 [======================>.......] - ETA: 0s - loss: 0.0040
Epoch 00001: val_loss improved from 0.01501 to 0.01455, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 1s 5ms/sample - loss: 0.0046 - val_loss: 0.0146
Epoch 2/50
150/171 [=========================>....] - ETA: 0s - loss: 0.0042
Epoch 00002: val_loss improved from 0.01455 to 0.01440, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 888us/sample - loss: 0.0043 - val_loss: 0.0144
Epoch 3/50
145/171 [========================>.....] - ETA: 0s - loss: 0.0041
Epoch 00003: val_loss improved from 0.01440 to 0.01425, saving model to model_checkpoints/saved_model.hdf5
171/171 [==============================] - 0s 895us/sample - loss: 0.0048 - val_loss: 0.0142
Epoch 4/50
160/171 [===========================>..] - E

...

150/179 [========================>.....] - ETA: 0s - loss: 0.0012
Epoch 00035: val_loss did not improve from 0.00279
179/179 [==============================] - 0s 776us/sample - loss: 0.0010 - val_loss: 0.0028
Epoch 36/50
150/179 [========================>.....] - ETA: 0s - loss: 9.9789e-04
Epoch 00036: val_loss did not improve from 0.00279
179/179 [==============================] - 0s 756us/sample - loss: 9.1770e-04 - val_loss: 0.0028
Epoch 37/50
150/179 [========================>.....] - ETA: 0s - loss: 9.9532e-04
Epoch 00037: val_loss did not improve from 0.00279
179/179 [==============================] - 0s 772us/sample - loss: 0.0010 - val_loss: 0.0028
Epoch 38/50
155/179 [========================>.....] - ETA: 0s - loss: 0.0014    
Epoch 00038: val_loss did not improve from 0.00279
179/179 [==============================] - 0s 747us/sample - loss: 0.0012 - val_loss: 0.0028
Epoch 39/50
150/179 [========================>.....] - ETA: 0s - loss: 5.7716e-04
Epoch 00039: val_loss did 

  0%|          | 10/2758 [01:30<6:52:01,  9.00s/it]

CPU times: user 102 ms, sys: 4.5 ms, total: 106 ms
Wall time: 126 ms
Train on 180 samples, validate on 20 samples
Epoch 1/50
130/180 [====================>.........] - ETA: 0s - loss: 0.0015   
Epoch 00001: val_loss did not improve from 0.00279
180/180 [==============================] - 1s 3ms/sample - loss: 0.0012 - val_loss: 0.0030
Epoch 2/50
150/180 [========================>.....] - ETA: 0s - loss: 0.0015    
Epoch 00002: val_loss did not improve from 0.00279
180/180 [==============================] - 0s 797us/sample - loss: 0.0013 - val_loss: 0.0030
Epoch 3/50
160/180 [=========================>....] - ETA: 0s - loss: 0.0014   
Epoch 00003: val_loss did not improve from 0.00279
180/180 [==============================] - 0s 738us/sample - loss: 0.0013 - val_loss: 0.0030
Epoch 4/50
155/180 [========================>.....] - ETA: 0s - loss: 0.0015   
Epoch 00004: val_loss did not improve from 0.00279
180/180 [==============================] - 0s 745us/sample - loss: 0.0015 - val_loss

Epoch 37/50
150/180 [========================>.....] - ETA: 0s - loss: 0.0011    
Epoch 00037: val_loss did not improve from 0.00279
180/180 [==============================] - 0s 756us/sample - loss: 0.0013 - val_loss: 0.0030
Epoch 38/50
160/180 [=========================>....] - ETA: 0s - loss: 7.0814e-04
Epoch 00038: val_loss did not improve from 0.00279
180/180 [==============================] - 0s 750us/sample - loss: 8.5888e-04 - val_loss: 0.0030
Epoch 39/50
  5/180 [..............................] - ETA: 0s - loss: 1.4425e-04

KeyboardInterrupt: 

In [46]:
daily_portfolio_weights

[[0.05,
  0.05,
  0.05000003802197807,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.500000005744744,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.05000007358533766,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.4999999644863063,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.05000016026692852,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.4999998716957945,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.05000030035694267,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.4999997262687899,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.05000172093742162,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.4999983013638237,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.050000500714416564,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.4999995634925719,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.05000429791846216,
  0.05,
  0.05,
  0.05,
  0.05,
  0.0500000006168396,
  0.49999571789940317,
  0.05,
  0.05000000025830422],
 [0.05,
  0.05,
  0.08111618895682647,
  0.05,
  0.05,
  0.05,
  0.05,
  0.05,
  0.46888387385026176,
  0.05,
  0.05],
 [0.05,
  0.05,
  0.12264

In [ ]:
"""
# Mini-batch gradient descent:

def fetch_batch(batch_instance, batch_size, features, log_returns, n_timesteps, look_ahead_time):
    
    """
    batch_instance: which batch we are in out of the total number of batches in the epoch (starts from 0)
    batch_size: the number of instances in each batch
    n_timesteps: length of series used for prediction (i.e. how many days we're predict off of)
    look_ahead_time: number of days in advance we will predict
    features: features matrix
    log_returns: response var matrix
    """
    
    # begin with empty arrays to which we will append 
    X = np.array([]) 
    y = np.array([])
    
    for i in range((batch_instance*batch_size), (batch_instance*batch_size) + batch_size):
        
        ### e.g.: range[0,20), range[20,40), range[40,60) etc if batch_size is 20 ###
        ### note that range function is NOT inclusive of last integer ###
        
        end_index = i + n_timesteps
        
        ### e.g.: end_index = 0 + 10 = 10 for first batch instance and n_timesteps = 10 ###
        
        # return (X_one:[n_timesteps,n_features], y_one:[look_ahead_time, n_assets]) starting on day 'i'
        X_one, y_one = sliding_window(features, log_returns, end_index, n_timesteps, look_ahead_time)
        
        # append X_one:[n_timesteps,n_features] to batch ndarray X
        X = np.append(X,X_one)
        # append y_one:[look_ahead_time, n_assets] to batch ndarray y
        y = np.append(y,y_one)
    
    # reshape to 3D with number_of_batch_instances x n_time_steps x n_features
    X = X.reshape(batch_size, n_timesteps, features.shape[1])
    # reshape to 2D with number_of_batch_instances x n_features
    y = y.reshape(batch_size, log_returns.shape[1])
    
    return X, y 
"""    

In [ ]:
"""
# start by iterating through epochs
for epoch in range(n_epochs):
    
    # then iterate through n_batches
    for batch_instance in range(n_batches):
        
        # fetch the batches
        X_batch, y_batch = fetch_batch(batch_instance, batch_size, features, 
                                       log_returns, n_timesteps, look_ahead_time)
        
        # fit batches to model
        model.fit(X_batch, y_batch, batch_size=batch_size)
        
"""       